In [ ]:
#!!!!! 执行以下语句之前，切记，先确保库存stock_age_cal中的负数转换正数，然后添加至每日出库daily_stockout_sell,同时注意修改时间。
# 确保daily_stockout_sell_0930里面时间大于等于2018-11-30

In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from dateutil.parser import parse

In [ ]:
# #数据库连接初始化设置
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="qaz!@wsx",
#     database="work"
# )
# print(mydb)

In [ ]:
#数据库连接初始化
mydb = mysql.connector.connect(
    host="rm-bp178mt2593im687xjo.mysql.rds.aliyuncs.com",
    user="root",
    passwd="KLxs8888",
    database="wdgj"
)
print(mydb)

In [ ]:
#SQL语句
sqlStock="SELECT * FROM stock_age_cal"
sqlSell ="SELECT * FROM daily_stockout_sell"

In [ ]:
#库存数据获取
mycursor=mydb.cursor()
mycursor.execute(sqlStock)
stockdata=mycursor.fetchall()

In [ ]:
#将数据转化为DaraFrame
stockdata=pd.DataFrame(stockdata)
stockdata.columns=['GoodsNO','GoodsName','Total_in','date']
stockdata=stockdata[['GoodsNO','GoodsName','Total_in','date']]

In [ ]:
stockdata.head()

In [ ]:
stockdata =stockdata.fillna(np.nan)

In [ ]:
stockdata.isnull().sum()

In [ ]:
stockdata['Total_in']=stockdata['Total_in'].fillna(0)

In [ ]:
stockdata.shape

In [ ]:
#删除小于等于0的数值
index = stockdata[stockdata['Total_in']<=0].index
stockdata=stockdata.drop(index=index,axis=0)

In [ ]:
stockdata.shape

In [ ]:
#按SKU分类汇总
stockdata_groupby = stockdata.groupby(['GoodsNO','GoodsName','date'],as_index=False)
stockdata_groupby=stockdata_groupby.aggregate(np.sum)

In [ ]:
stockdata_groupby['cal_time']=''

In [ ]:
stockdata_groupby.head()

In [ ]:
stockdata_groupby.shape

In [ ]:
#清除库存为0的数据
index = stockdata_groupby[stockdata_groupby['Total_in']==0].index
stockdata_groupby=stockdata_groupby.drop(index=index,axis=0)

In [ ]:
stockdata_groupby.shape

In [ ]:
stockdata_groupby['date'].head()

In [ ]:
# stockdata_groupby.head()

In [ ]:
# stock_change_data_old = pd.read_csv('stock_change_datac.csv',header = 0)
# stock_change_data_old.shape
# df=[stockdata_groupby,stock_change_data_old]
# stockdata_groupby=pd.concat(df)
# stockdata_groupby.shape

In [ ]:
stockdata_groupby['date'].head()

In [ ]:
#获取每日销量数据
mycursor=mydb.cursor()
mycursor.execute(sqlSell)
stocksell=mycursor.fetchall()

In [ ]:
stocksell=pd.DataFrame(stocksell)
stocksell.columns=['SKU','name','total','date']
stocksell=stocksell[['SKU','name','total','date']]

In [ ]:
stocksell.head()

In [ ]:
stocksell =stocksell.fillna(np.nan)

In [ ]:
stocksell.isnull().sum()

In [ ]:
stocksell.shape

In [ ]:
stocksell_groupby = stocksell.groupby(['SKU','name','date'],as_index=False)

In [ ]:
stocksell_groupby

In [ ]:
stocksell_groupby=stocksell_groupby.aggregate(np.sum)

In [ ]:
stocksell_groupby.shape

In [ ]:
index = stocksell_groupby[stocksell_groupby['date']<parse('2017-12-31').date()].index
stocksell_groupby=stocksell_groupby.drop(index=index,axis=0)

In [ ]:
stocksell_groupby.head()

In [ ]:
stocksell_groupby.shape

In [ ]:
#按天分类
stocksell_groupby_date = stocksell_groupby.groupby(['date'],as_index=False)

In [ ]:
stocksell_groupby_date

In [ ]:
#按天获取对应数据
stocksell_groupby_date.get_group('2019-03-30').head()

In [ ]:
# stockdata_groupby

In [ ]:
#获取特定天的SKU，对应出库，构造SKU：Total字典
def get_sku_code(date):
    try:
        df=stocksell_groupby_date.get_group(date)
    except:
        df=[]
    l=len(df)
    dict_new={}
    for i in range(l):
        dict_new[df['SKU'].iloc[i]]=df['total'].iloc[i]
    return dict_new

In [ ]:
#计算库存变化，按照先进先出原则
def sku_stock_delete(time,stock_change):
    sell_dict=get_sku_code(time)#获取特定某一天的SKU:Total
    total=0
    a=0
    print(len(sell_dict))
    for key in sell_dict:
        total+=1
#         print(total)
        sell=sell_dict[key]
        #获取满足条件的对应所在行的index
        stock_index=stock_change[(stock_change['date']<=parse(time).date())&(stock_change['GoodsNO']==key)\
                                     &(stock_change['Total_in']>0)].sort_values(by ='date',ascending=True)['Total_in'].index
        #遍历每个index，按条件扣减对应的库存
        for i in stock_index:
            stock=stock_change['Total_in'].loc[i]
            if sell>=stock:
                stock_change['Total_in'].loc[i]=0
                sell=sell-stock
#                 stock_change['stock_time'].loc[i]=time
            else:
                stock_change['Total_in'].loc[i]=stock-sell
                sell=0
#                 stock_change['stock_time'].loc[i]=time
                break
        a=a+sell
    print(total)
    index_0 = stock_change[stock_change['Total_in']==0].index
    stock_change=stock_change.drop(index=index_0,axis=0)
    return stock_change,a

In [ ]:
#重新创建一个DataFrame，复制一份stockdata_groupby,用来确保在调用函数的过程中不改变stockdata_groupby
stock_change_data=pd.DataFrame()
stock_change_data=pd.concat([stock_change_data,stockdata_groupby])

In [ ]:
#构造存放计算后每天的库存数据表格
stock_age_cal=pd.DataFrame()
t=0

In [ ]:
starttime = datetime.now()
startdate=date(2019,3,1)
for i in range(31):
    start=startdate + timedelta(i)
    print(start)
    stock_change=sku_stock_delete(str(start),stock_change_data)#调用函数计算库存占用
    stock_change[0]['cal_time']=start#标记库存占用计算日期
    stock_age_cal=pd.concat([stock_age_cal,stock_change[0][stock_change[0]['date']<=start-timedelta(60)]])#筛选60天前的数据
    t+=stock_change[1]
endtime=datetime.now()
print ((endtime - starttime).seconds/60/60)


In [ ]:
stock_age_cal.to_csv('stock_age_cal.csv',header=True,index=False)

In [ ]:
stock_change_data.to_csv('stock_change_data.csv',header=True,index=False)

In [ ]:
stock_age_cal=pd.read_csv('stock_age_cal.csv',header = 0)

In [ ]:
stock_age_cal.head()

In [ ]:
#构造新的表格复制一份stock_age_cal
stock_age_cal_copy=pd.DataFrame()
stock_age_cal_copy=pd.concat([stock_age_cal_copy,stock_age_cal])

In [ ]:
# stock_age_cal.head()

In [ ]:
# stock_age_cal1=stock_age_cal1[stock_age_cal1['cal_time']<'2018-02-11']

In [ ]:
# stock_age_cal_copy=pd.concat([stock_age_cal1,stock_age_cal])

In [ ]:
# stock_age_cal_copy=stock_age_cal_copy[stock_age_cal_copy['cal_time']<='2018-12-31']

In [ ]:
# stock_age_cal_copy.to_csv('stock_age_cal12.csv',header=True,index=False)

In [ ]:
stock_age_cal_copy['date']=pd.to_datetime(stock_age_cal_copy['date'])
stock_age_cal_copy['cal_time']=pd.to_datetime(stock_age_cal_copy['cal_time'])

In [ ]:
stock_age_cal_copy['stockAge']=stock_age_cal_copy['cal_time']-stock_age_cal_copy['date']

In [ ]:
stock_age_cal_copy.head()

In [ ]:
#选择库龄大于60天的SKU
stock_age_cal_copy_60=stock_age_cal_copy[(stock_age_cal_copy['stockAge']>=timedelta(60))&(stock_age_cal_copy['cal_time']>'2018-11-30')]

In [ ]:
stock_age_cal_copy_60.head()

In [ ]:
# #获取商品固定价格
# goodslist=pd.read_csv('C:\\Users\\Administrator\\PycharmProjects\\test\\g_goods_goodslist.csv',\
#                       header = 0,dtype=object,engine='python',encoding='utf-8')
# goodslist_short=goodslist[['GoodsNO','FixCostPrice']]
# goodslist_short=goodslist_short.drop_duplicates(["GoodsNO"]) #SKU去重

In [ ]:
mydb = mysql.connector.connect(
    host="rm-bp178mt2593im687xjo.mysql.rds.aliyuncs.com",
    user="root",
    passwd="KLxs8888",
    database="wdgj"
)

In [ ]:
sql_goodslist='SELECT GoodsNO,FixCostPrice,BarCode FROM g_goods_goodslist'
mycursor=mydb.cursor()
mycursor.execute(sql_goodslist)
goodslist=mycursor.fetchall()

In [ ]:
goodslist = pd.DataFrame(goodslist)

In [ ]:
goodslist.columns=['GoodsNO','FixCostPrice','BarCode']

In [ ]:
goodslist = pd.DataFrame(goodslist)
goodslist_short=goodslist[['GoodsNO','FixCostPrice','BarCode']]
goodslist_short=goodslist_short.drop_duplicates(["GoodsNO"])

In [ ]:
#匹配库存和商品列表
data_merge=pd.merge(stock_age_cal_copy_60,goodslist_short,on=['GoodsNO'],how='left')
data_merge['FixCostPrice']=data_merge['FixCostPrice'].apply(pd.to_numeric) # 将FixCostPrice转换成数值类型

In [ ]:
data_merge=data_merge.fillna(np.nan)

In [ ]:
data_merge.isnull().sum()

In [ ]:
data_merge['FixCostPrice']=data_merge['FixCostPrice'].fillna(0)

In [ ]:
#计算每个SKU的库存占用费
data_merge['stockDelayCost']=data_merge['Total_in']*data_merge['FixCostPrice']*0.05/30

In [ ]:
data_merge=data_merge[(data_merge['stockAge']>=timedelta(60))&(data_merge['cal_time']>'2018-12-31')]

In [ ]:
data_merge.shape

In [ ]:
#data_merge.to_excel('data_merge.xlsx',header=True,index=False)

In [ ]:
data_merge['cal_time_month']=data_merge.cal_time.map(lambda x: x.strftime('%Y-%m')) # 取月份数据

In [ ]:
#分SKU，月份汇总
data_merge_groupby=data_merge.groupby(['BarCode','GoodsName','cal_time_month'],as_index=False)
data_merge_groupby=data_merge_groupby.aggregate(np.sum)

In [ ]:
data_merge_groupby.head()

In [ ]:
#获取小组信息并去重
GoodsInfo =pd.read_csv('C:\\Users\\Administrator\\PycharmProjects\\test\\goodsinfo0402.csv',header = 0,engine='python')
GoodsInfo.columns=['GoodsNO','GoodsName','BarCode','Company','Dept','tamllTax','Group','Holder','GradeTwo','GradeThird','GradeForth','SPU']
GoodsInfoShort=GoodsInfo[['BarCode','Company','Dept']]
GoodsInfoShort=GoodsInfoShort.drop_duplicates(['BarCode'])

In [ ]:
#匹配部门
data_merge_dept=pd.merge(data_merge_groupby,GoodsInfoShort,on=['BarCode'],how='left')

In [ ]:
data_merge_dept.columns

In [ ]:
data_merge_dept=data_merge_dept.fillna(np.nan)
data_merge_dept.isnull().sum()

In [ ]:
data_merge_groupby.columns

In [ ]:
data_merge_dept=data_merge_dept[['BarCode', 'GoodsName', 'cal_time_month', 'Total_in', 'stockDelayCost','Company', 'Dept']]

In [ ]:
data_merge_dept.to_excel('data_merge_dept.xlsx',header=True,index=False)

In [ ]:
#按月，公司，部门汇总
data_merge_dept_gb = data_merge_dept.groupby(['Company','Dept','cal_time_month'],as_index=False)
data_merge_dept_gb=data_merge_dept_gb.aggregate(np.sum)
data_merge_dept_gb=data_merge_dept_gb[['Company','Dept','cal_time_month','stockDelayCost']]

In [ ]:
data_merge_dept_gb.head()

In [ ]:
data_merge_dept_gb.to_excel("2019库存占用带小组9.xlsx",header=True,index=False)

In [ ]:
# stocksell_groupby[stocksell_groupby['date']<=parse('2018-12-31').date()].sort_values(by ='date',ascending=True).head()

In [ ]:
stock_change_data = pd.read_csv('stock_change_data.csv',header = 0)
stock_change_data['date']=pd.to_datetime(stock_change_data['date'])

In [ ]:
#校核检验结果
result=stockdata_groupby[stockdata_groupby['date']<=parse('2019-03-31').date()].groupby(['date'],as_index=False)
result=result.aggregate(np.sum)#初始库存总数
result1=stock_change_data[stock_change_data['date']<=parse('2019-03-31').date()].groupby(['date'],as_index=False)
result1=result1.aggregate(np.sum)#最终库存总数
result2=stocksell_groupby[stocksell_groupby['date']<=parse('2019-03-31').date()].groupby(['date'],as_index=False)
result2=result2.aggregate(np.sum)#出库总数
result.Total_in.sum()

In [ ]:
#检验是否计算正确，为零则表示准确无误
result1.Total_in.sum()-(result.Total_in.sum()-result2.total.sum()+t)

In [ ]:
result1.Total_in.sum()

In [ ]:
result2.total.sum()

In [ ]:
t

In [ ]:
startdate=date(2019,3,1)
for i in range(31):
    start=startdate + timedelta(i)
    print(start)